In [1]:
import pandas as pd
import os
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import pymp
import time
import qgrid
from sklearn.metrics import roc_auc_score
from multiprocessing import Manager, cpu_count
from projects.feed.rank.utils.doc_info.KV import KV

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/tuwen/8'
hour = 2020050822
info_path = f'{info_dir}/{hour}/valid.csv'
print(info_path)
df = pd.read_csv(info_path)
# df2 = pd.read_csv()
# df = df.rename({'duration':'dur'})
df['dur'] = df.duration
df['mark'] = 'tuwen'
info_dir = f'/search/odin/publicData/CloudS/libowei/rank_online/infos/video/8'
info_path = f'{info_dir}/{hour}/valid.csv'
df2 = pd.read_csv(info_path)
df2['dur'] = df.duration
df2['mark'] = 'video'
df = pd.concat([df, df2])
df = df[df.product_data=='newmse']
df.head()

/search/odin/publicData/CloudS/libowei/rank_online/infos/tuwen/8/2020050822/valid.csv


,doc_topic,abtestid,article_page_time,coldstart_refresh_num,did,distribution,docid,duration,id,impression_time,...,click,weight,cold,product_data,pred,pred_click,pred_dur,hour,dur,mark
50,0,2,1586852566,1,7334023469134679818,0,20414g0l4G4fL2,0,00000000-5a0a-9d0f-512a-c0486bc5b3c9\t20414g0l...,1588945865,...,0,1.0,0,newmse,0.132690,0.376763,0.354046,2020050822,0,tuwen
51,0,1,1588932695,3,-8639180447363974982,0,20508i0x4b926J,0,ffffffff-f1b3-e10f-ffff-ffffeec6e737\t20508i0x...,1588946512,...,0,1.0,0,newmse,0.024684,0.163107,0.171918,2020050822,0,tuwen
52,0,14,1588749211,1,4420956366464759396,0,20506g0l0NQKbo,146,ffffffff-9f81-581d-ffff-fffff1721281\t20506g0l...,1588946189,...,1,1.0,0,newmse,0.060682,0.263637,0.209410,2020050822,146,tuwen
53,0,4,1587975586,7,7992449200751595495,0,20427m0l1ZdcHG,0,00000000-7b0a-2764-14d6-a2757c1151fc\t20427m0l...,1588948194,...,0,1.0,0,newmse,0.042883,0.191227,0.428480,2020050822,0,tuwen
54,0,16,1588745148,3,9029453680989744392,0,open_doc_prod28146024,0,ffffffff-9ade-7f35-ffff-ffff82522d10\topen_doc...,1588947579,...,0,1.0,0,newmse,0.029601,0.165585,0.288455,2020050822,0,tuwen


In [4]:
df['click'] = (df.dur != 0).astype(int)

In [5]:
kv = KV()
titles = kv.get_titles(list(set(df.docid.values)))
df['title'] = df.docid.apply(lambda x: titles.get(x, ''))

get_titles: 100%|##########| 2600/2600 [01:50<00:00, 23.46it/s]


In [6]:
pd.set_option('display.max_rows', 1000)
pd.set_option("display.precision", 8)

In [7]:
if 'article_page_time' not in df.columns:
  df.article_page_time = df.impression_time
df['page_time'] = (df.impression_time - df.article_page_time) / 60 / 60 / 24

In [8]:
df['impression'] = df.mid + '\t' + df.impression_time.astype(str)

In [9]:
mean_duration = df[df.dur > 0].dur.mean()

In [10]:
mean_duration

81.48809885247468

In [11]:
from projects.feed.rank.src import util
df['cold'] = df.rea.apply(lambda x: int(x) in util.cold_starts)
# df = df[df.product_data=='sgsapp']

In [12]:
import math
df.pred = df.pred.apply(lambda x: math.sqrt(x))

In [13]:
cold_mids = set(df[df.cold==True].mid.values)

In [14]:
len(set(df.mid.values))

199913

In [15]:
len(cold_mids)

31230

In [16]:
def debug(mid=None, cold=False):
  if mid is None:
    if not cold:
      mid = df.mid.sample(1).values[0]
    else:
      mid = df[df.mid.isin(cold_mids)].mid.sample(1).values[0]
  print('mid:', mid)
  cols = ['position', 'title', 'mark', 'dur', 'ori_lr_score', 'ol_pred', 'ol_pred_click', 'ol_pred_dur', 'docid', 'model_ver']
#   if 'hour' in df.columns:
#     cols += ['hour']
  d = df[df.mid==mid]
  start_time = d.impression_time.min()
  d['time'] = d.impression_time.apply(lambda x: int((x - start_time) / 60))
#   cols += ['time']
  d = d.sort_values(['impression_time', 'position'])[cols].reset_index(drop=True)
#   return d.sort_values(['ori_lr_score'], ascending=False)[cols].reset_index(drop=True)
  return d

In [17]:
debug("e77c1587466574307|a0fb78")

mid: e77c1587466574307|a0fb78


,position,title,mark,dur,ori_lr_score,ol_pred,ol_pred_click,ol_pred_dur,docid,model_ver


In [18]:
len(set(df.impression.values)) 

521837

In [19]:
len(set(df.mid.values))

199913

In [20]:
len(set(df.impression.values))  / len(set(df.mid.values))

2.6103204894128944

In [21]:
len(set(df[df.dur!=0].mid.values))

144318

In [22]:
len(set(df[df.dur!=0].mid.values)) / len(set(df.mid.values))

0.7219040282522897